  # Гриша Кузнецов

# Обработка mystem
Тут я импортирую нужные модули
1) Использую консольный mystem с параметрами -ln. Так я выведу только леммы, по одной на каждой строке
2) Из них я делаю массив лемм
3) Затем записываю этот массив в json с помощью dump

In [1]:
import os
import timeit
import time
import json
from string import punctuation 
from collections import Counter
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
morph = MorphAnalyzer()


json_lemms = []
%time os.system('.\mystem.exe -inc --format json book.txt bookout.json')
with open('bookout.json', 'r', encoding='utf-8') as f:
    lemms = f.readlines()
    for lemm in lemms:
        json_lemms.append(json.loads(lemm))
with open('lemms_out.json', 'w', encoding='utf-8') as f:
    json.dump(json_lemms, f, ensure_ascii=False)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Гриша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Wall time: 4.94 s


# Обработка pymorphy
1) Открываю исходный текст книги, привожу в опрятный вид и токенизирую его. 

In [2]:
with open('book.txt', 'r', encoding='utf-8') as f:
    booktext = f.read()
for i in punctuation:
    booktext = booktext.replace(i, '')
%time tokens = (word_tokenize(booktext))


Wall time: 613 ms


### Сохранение в json
1) Ищу аннотацию для каждого токена
2) Достаю оттуда часть, содержащее слово, массив тегов, вероятность
3) Чтобы засунуть теги в JSON, я привел tag к формату str (с помощью str(), не знаю, насколько это правильно, но работает)
4) Создаю словарь, где ключ - слово, значение - массив его характеристик

In [3]:
%%time
anno_d = {}
for token in tokens:
    annotation = morph.parse(token)
    parse = annotation[0]
    tag = parse.tag
    tagstr = str(tag)
    tagstr = tagstr.split(',')
    anno_d[parse.word] = tagstr, parse.normal_form, parse.score
with open('tags.json', 'w', encoding='utf-8') as f:
    json.dump(anno_d, f, ensure_ascii=False)

Wall time: 45.2 s


# Какую долю слов составляет каждая часть речи? (Например, для глагола - это количество глаголов, деленное на общее число слов в тексте)
1) Тут я делаю массивы для частей речи, глаголов и наречий
2) Прохожусь по всем токенам, паршу, беру содержательную часть Parse как в предыдущем задании
3) Отбираю глаголы и наречия в свои массивы
4) Делаю общий массив всех значений Part Of Speech
5) Считаю токены для процентных соотношений 

In [ ]:
pos_a = []
token_count = 0
verbs = []
adverbs = []
for token in tokens:
    annotation = morph.parse(token)
    parse = annotation[0]
    if parse.tag.POS == 'VERB':
        verbs.append(parse.word)
    if parse.tag.POS == 'ADVB':
        adverbs.append(parse.word)
    if parse.tag.POS is not None:
        pos_a.append(parse.tag.POS)
        token_count += 1
pos_c = Counter(pos_a)
for key, value in pos_c.items():
    percent = (value / token_count) * 100
    print(key, round(percent, 2), '%') 

# Найдите топ-20 (по частотности) глаголов и наречий
### Глаголы
Делаю Counter по списку глаголов и вывожу их вместе с количеством употреблений в тексте

In [ ]:
verbs_c = Counter(verbs)
adverbs_c = Counter(adverbs)
print('TOP-20 verbs: ')
for i in verbs_c.most_common(20):
    print(i[0], i[1])

### Наречия
Делаю то же самое для наречий

In [ ]:
print('TOP-20 adverbs: ')
for i in adverbs_c.most_common(20):
    print(i[0], i[1])

# Биграммы и триграммы
### Биграммы
1) Использую функцию для биграмм, сразу делая массив
2) Из массива делаю Counter и вывожу биграмму с количеством взождений в тексте

In [ ]:
bigrams = list(nltk.bigrams(tokens))
bi_c = Counter(bigrams)
for i in bi_c.most_common(25):
    print(i[0], i[1])

### Триграммы
Делаю то же самое для триграмм

In [ ]:
trigrams = list(nltk.trigrams(tokens))
tri_c = Counter(trigrams)
for i in tri_c.most_common(25):
    print(i[0], i[1])